In [17]:
from copy import deepcopy

import numpy as np
import open3d as o3d
import trimesh

In [42]:
mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\data\Dancer\meshes/dancer_fr0010.obj')
print(mesh)
testmesh = o3d.geometry.TriangleMesh()
testmesh.vertices = mesh.vertices
testmesh.triangles = mesh.triangles
testmesh.compute_vertex_normals()

print(testmesh.has_textures())
print(testmesh)
#o3d.visualization.draw_geometries([testmesh])

TriangleMesh with 20913 points and 39357 triangles.
False
TriangleMesh with 20913 points and 39357 triangles.


In [97]:
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(testmesh, 10000, boundary_weight= 8000)
print(decimated_mesh)
decimated_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([decimated_mesh])

TriangleMesh with 6168 points and 10000 triangles.


simplify_vertex_clustering is not good for subdivide surface fitting, simplify_quadric_decimation is much better
notes that boundary_weight will influence the result of decimated mesh, inappropriate setting leads to holes in the mesh.

In [138]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh()
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([subdivided_mesh])
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [99]:
fitting_mesh = subdivide_surface_fitting(decimated_mesh, mesh, 1)
fitting_mesh.compute_vertex_normals()
fitting_mesh.paint_uniform_color([0.7, 0.7, 0.7])
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([fitting_mesh])

TriangleMesh with 22314 points and 40000 triangles.
22314


In [102]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(5,10):
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\output/deformed_{i:03}.obj')
    mesh.compute_vertex_normals()
    #print(mesh)
    deformed_meshes.append(mesh)
    all_meshes += mesh
o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())

TriangleMesh with 98617 points and 196912 triangles. True
PointCloud with 98617 points. True


In [123]:
GoF = 5
key = 0
fitting_meshes = []
for i in range(0,GoF):
    if i==key:
        continue
    decimated_mesh_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(deformed_meshes[i], 10000, boundary_weight= 8000)
    print(decimated_mesh_i)
    fitting_mesh_i = subdivide_surface_fitting(decimated_mesh_i, deformed_meshes[key], 1)
    print(fitting_mesh_i)
    fitting_mesh_i.compute_vertex_normals()
    fitting_meshes.append(fitting_mesh_i)
fitting_meshes.append(deformed_meshes[key])

TriangleMesh with 5035 points and 9999 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 5038 points and 10000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 5042 points and 9999 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 5041 points and 10000 triangles.
TriangleMesh with 20087 points and 40000 triangles.
TriangleMesh with 20087 points and 40000 triangles.


In [132]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(0,GoF):
    all_meshes += fitting_meshes[i]
#o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pre_reference_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(reference_pcd, depth=9, linear_fit=True)
print(mesh)
pre_reference_mesh.compute_vertex_normals()

pre_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
o3d.visualization.draw_geometries([pre_reference_mesh])

TriangleMesh with 100051 points and 199372 triangles. True
PointCloud with 100051 points. True
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 100051 / 18528
[Open3D DEBUG] #   Got kernel density: 0.0120001 (s), 433.355 (MB) / 433.355 (MB) / 603 (MB)
[Open3D DEBUG] #     Got normal field: 0.236 (s), 435.086 (MB) / 435.086 (MB) / 603 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 7.589807e-06 / 7.593678e-01
[Open3D DEBUG] #       Finalized tree: 0.151 (s), 464.398 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0870001 (s), 447.199 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] #Set point constraints: 0.027 (s), 445.844 (MB) / 464.398 (MB) / 603 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 839231 / 955032 / 4089
[Open3D DEBUG] Memory Usage: 445.844 MB
[Open3D DEBUG] # Linear system solved: 0.344 (s), 471.207 (MB) / 471.207 (MB) / 603 (MB)
[Open3D DEBUG] Got average: 0.00300002 (s), 450.488 (MB) / 471.207 (MB) / 603 (MB

In [133]:
reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(pre_reference_mesh, 40000, boundary_weight= 8000)
print(reference_mesh)
reference_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([reference_mesh])

TriangleMesh with 20008 points and 40000 triangles.


In [136]:
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [21]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)
print(load_reference_mesh)
reference_vertices = np.array(load_reference_mesh.vertices)
load_reference_mesh.compute_vertex_normals()
#o3d.visualization.draw_geometries([load_reference_mesh])
load_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
print(reference_vertices)

TriangleMesh with 20002 points and 40000 triangles.
[[ 0.0791106   0.90974599 -0.477777  ]
 [ 0.0861452   0.90974599 -0.48245999]
 [ 0.0861452   0.905756   -0.48109001]
 ...
 [-0.127333    1.02546     0.26905501]
 [ 0.174541   -0.25041699  0.27527499]
 [-0.0575035   0.39899501  0.31645399]]


In [22]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh


In [24]:
reference_mesh_trimesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj')
print(reference_mesh_trimesh)
np.array(reference_mesh_trimesh.vertices)


TriangleMesh with 20002 points and 40000 triangles.


array([[ 0.0905365,  0.909746 , -0.482768 ],
       [ 0.0861452,  0.909746 , -0.48246  ],
       [ 0.0861452,  0.905756 , -0.48109  ],
       ...,
       [ 0.225804 , -0.494819 ,  0.35502  ],
       [ 0.225804 , -0.486839 ,  0.357004 ],
       [ 0.245755 , -0.494819 ,  0.354405 ]])

## deal with reorder problem

In [25]:
mesh = deepcopy(load_reference_mesh)
true_colors = np.array(mesh.vertex_colors,copy=True)
vertex_reindexer = np.arange(len(mesh.vertices))
mesh.vertex_colors = o3d.utility.Vector3dVector((vertex_reindexer / vertex_reindexer[-1]).reshape(-1,1)[:,[0,0,0]])
o3d.io.write_triangle_mesh(
    r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj',mesh,
    write_ascii=False,write_vertex_normals = False, write_vertex_colors = True
)
hardened_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj',enable_post_processing = False)
remaining_reordered_vertices = (np.asarray(hardened_mesh.vertex_colors)[:,0] * vertex_reindexer[-1] + 0.5).astype(int)
hardened_mesh.vertex_colors = o3d.utility.Vector3dVector(true_colors[remaining_reordered_vertices])
# reorder all additional data linked to the vertices using the remaining_reordered_vertices
# store the hardened_mesh using the final filename

[Open3D WARNING] Write OBJ can not include triangle normals.


In [26]:
vertices = np.array(hardened_mesh.vertices)
print(vertices)

[[ 0.0791106   0.90974599 -0.477777  ]
 [ 0.0861452   0.90974599 -0.48245999]
 [ 0.0861452   0.905756   -0.48109001]
 ...
 [-0.127333    1.02546     0.26905501]
 [ 0.174541   -0.25041699  0.27527499]
 [-0.0575035   0.39899501  0.31645399]]


In [141]:
reconstruct_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_005.obj')
original_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(reconstruct_dancer_5)
print(original_dancer_5)
reconstruct_dancer_5.compute_vertex_normals()
original_dancer_5.compute_vertex_normals()
o3d.visualization.draw_geometries([reconstruct_dancer_5])

TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.


In [142]:
decimated_reconstructed_5 = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_5, 10000, boundary_weight= 8000)
print(decimated_reconstructed_5)
fitting_mesh_dancer_5 = subdivide_surface_fitting(decimated_reconstructed_5, original_dancer_5, 1)
print(fitting_mesh_dancer_5)
fitting_mesh_dancer_5.compute_vertex_normals()

o3d.visualization.draw_geometries([fitting_mesh_dancer_5])

TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles.


In [143]:
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_5.obj', fitting_mesh_dancer_5, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

[Open3D WARNING] Write OBJ can not include triangle normals.


True